In [3]:
import nibabel as nib
from nibabel.processing import resample_from_to
import numpy as np
import torch
ct_path = '/Users/keyi/Desktop/MA/HECKTOR/imagesTr/CHUM-001__CT.nii.gz'
pet_path = '/Users/keyi/Desktop/MA/HECKTOR/imagesTr_resampled/CHUM-001__PT.nii.gz'
label_path = '/Users/keyi/Desktop/MA/HECKTOR/labelsTr_resampled/CHUM-001.nii.gz'
ana_path = '/Users/keyi/Desktop/MA/HECKTOR/totalsegmentator/CHUM-001__interpolated_ANASEG.nii.gz'
ct = nib.load(ct_path)
pet = nib.load(pet_path)
label = nib.load(label_path)
ana = nib.load(ana_path)
print(ct.shape)
print(pet.shape)
print(label.shape)
print(ana.shape)


print(np.unique(label.get_fdata()))
print(np.unique(ana.get_fdata()))






(512, 512, 91)
(512, 512, 91)
(512, 512, 91)
(512, 512, 91)
[0. 1.]
[  0.  10.  15.  16.  17.  32.  44.  52.  54.  69.  70.  71.  72.  73.
  74.  79.  86.  90.  91.  92. 118.]


In [ ]:
from monai.transforms import (
    EnsureChannelFirstd,
    LoadImaged,
    RandFlipd,
    RandRotate90d,
    SignalFillEmptyd,
    Compose,
    ConcatItemsd,
    ToTensord,
    RandShiftIntensityd,
    CropForegroundd,
    Orientationd,
    Spacingd,
    Transform,
    MapTransform,
    NormalizeIntensityd,
    LambdaD,
    RandGaussianNoised,
    RandGaussianSmoothd,
    RandCropByPosNegLabeld,
)
keys = ["image", "label"]
class ClipPercentiles(Transform):
    def __init__(self, lower_percentile:float, upper_percentile:float) -> None:
        super().__init__()
        self.lower_percentile = lower_percentile
        self.upper_percentile = upper_percentile

    def __call__(self, img:torch.Tensor):
        lower_threshold =   np.quantile(img,self.lower_percentile)
        upper_threshold = np.quantile(img,self.upper_percentile)
        return torch.clamp(img, min=lower_threshold, max=upper_threshold)
class ClipPercentaged(MapTransform):
    def __init__(self, keys, lower_percentile: float, upper_percentile: float) -> None:
        super().__init__(keys)
        self.clip_transform = ClipPercentiles(lower_percentile, upper_percentile)
    def __call__(self, data):
        d = dict(data)
        for key in self.keys:
            d[key] = self.clip_transform(d[key])
        return d
transforms = Compose([
    LoadImaged(keys=keys),
    EnsureChannelFirstd(keys=keys),
    Orientationd(keys=keys, axcodes="RAS"),
    Spacingd(keys=keys, pixdim=[0.977, 0.977, 3.27], mode=("bilinear", "nearest")),
    CropForegroundd(keys=keys, source_key="image", select_fn=lambda x: x > (-800 - 31.793294743021132)/41.282285307412266),
    ClipPercentaged(
            keys=["image"],
            lower_percentile=0.5,
            upper_percentile=0.995
        ),
        
        #Clip according to percentile in PET 
        #(include more varienty: Hypothesis: We should not clip off the spikes too much)
        
       
        #Noramlize the CT image. This has been done according to the nnUnet dataset fingerprint style
        NormalizeIntensityd(
            keys=["image"],
            subtrahend=31.793294743021132,
            divisor=41.282285307412266
        ),
        #Normalize PET image. This has been calculated according to the nnUnet dataset fingerprint style
        
        
    RandCropByPosNegLabeld(
            keys=keys,
            label_key="label",
            spatial_size=(64, 64, 64),
            pos=1,
            neg=0,
            num_samples=50,
            image_key="image",
            image_threshold=(-800 - 31.793294743021132)/41.282285307412266,
            allow_smaller=False,
            )
])
data = {"image": ct_path, "label": label_path}
transformed = transforms(data)
print(len(transformed))
print(transformed[0]["image"].shape)
print(transformed[0]["label"].shape)


In [1]:
import numpy as np
score = [0.6443,0.6546,0.6691,0.5980,0.5523]
print(np.mean(score))


0.6236599999999999


In [3]:
a = 0.5257 - 0.4931
print(f"the difference is {a*100}%")

the difference is 3.2599999999999962%
